In [11]:
import pandas as pd
import numpy as np
from itertools import combinations
from tqdm import tqdm
import time
import random

In [12]:
def single_day_data_process(path, angles):
    """
    compute metrics for single day measurements
    
    Args:
    ====
    path: input data directory, string
    angles: selected angles used for computation, list
    
    Returns:
    ====
    X: global mean, float
    R: avergae range for selected angles, float
    mean_df: mean for each selected angles, 1D array
    """    
    day = pd.read_csv(path, skiprows=43, usecols= ['Source Slot', 'Job Cycle', 'ESFQR Angle (deg)', 'ESFQX Value (nm)'])
    day = day.dropna()
    day = day[day['Source Slot']==1]
    day = day[(day['Job Cycle']== 1) | (day['Job Cycle']== 2) | (day['Job Cycle']== 3)]
    day = day[day['ESFQR Angle (deg)'].isin(angles)]
    
    X = np.mean(day['ESFQX Value (nm)'])
    
    max_v = day.groupby(by='ESFQR Angle (deg)')['ESFQX Value (nm)'].max()
    min_v = day.groupby(by='ESFQR Angle (deg)')['ESFQX Value (nm)'].min()
    mean_df = day.groupby(by='ESFQR Angle (deg)')['ESFQX Value (nm)'].mean()
    R = np.mean(max_v-min_v)
    
    return X, R, mean_df
    

In [13]:
def data_process(paths, angles):
    """
    process 2-day data for given angles, to compute matching related metrics
    
    Args:
    ===
    paths: list of saving paths for each of 3day measurements, list of strings
    angles: selected angles for computation, list
    
    Returns:
    ====
    EV, AV, RR, NDC: desired metrics reflecting tool health
    """
    
    X1, R1, mean_df1 = single_day_data_process(paths[0], angles)
    X2, R2, mean_df2 = single_day_data_process(paths[1], angles)
    #X3, R3, mean_df3 = single_day_data_process(paths[2], angles)
    
    #R = (R1 + R2 + R3)/3
    R = (R1 + R2)/2
    K1 = 0.5908
    EV = R * K1
    
    K2 = 0.7071
    X_diff = max(X1, X2) - min(X1, X2)
    nr = 30
    if (X_diff*K2)**2-(EV**2/nr)>0:
        AV = np.sqrt((X_diff*K2)**2-(EV**2/nr))
    else: AV = 0
    
    RR = np.sqrt(EV**2 + AV**2)
    
    K3 = 0.3146
    
    mean_df = (mean_df1 + mean_df2)/2
    Rp = np.max(mean_df)-np.min(mean_df)
    
    NDC = 1.41*(Rp*K3/RR)
    #print(X_diff, EV, (X_diff*K2)**2-(EV**2/nr))
      
    return EV, AV, RR, NDC

**Finding the optimal angle selections for ESFQR 1EE**

In [14]:
#ESFQR SHR 1EE
paths = [r'D:\PWG_data\2019-10-1\Gauge-1-2-3-ESFQx_72Sector_1EE_SHR-191001132242.csv',
        r'D:\PWG_data\20181014\Gauge-1-2-3-ESFQx_72Sector_1EE_SHR-181014123620.csv']

In [15]:
valid_angles = list(range(0,360, 5))
#remove_ls = [0,135,225,70,100,105,115,125,155,160,165,170,180,190,250,270,275,295,310,335,355]

#for num in remove_ls:
#    valid_angles.remove(num)

In [16]:
valid_angles = [15,25,35,45,55,65,80,95,140,145,150,215,220,265,290,300,325]
len(valid_angles)

17

In [17]:
#random sampling
n = 0
df_combination_rand = pd.DataFrame()
angles_com_rand = []
for i in tqdm(range(1*10**3)):
    angle = random.sample(valid_angles,10) 
    EV, AV, RR, NDC = data_process(paths, angle)
    #df_combination_rand.loc[n,'angle']=angle
    df_combination_rand.loc[n,'EV']=EV
    df_combination_rand.loc[n,'AV']=AV
    df_combination_rand.loc[n,'RR']=RR
    df_combination_rand.loc[n,'NDC']=NDC
    angles_com_rand.append(angle)
    n += 1

df_combination_rand['angles'] = angles_com_rand

100%|██████████████████████████████████████| 1000/1000 [00:35<00:00, 27.93it/s]


In [18]:
df_combination_rand.to_csv(r'D:\PWG_data\s5_c1-3_ESFQR_1EE_SHR.csv')

In [19]:
df_combination_rand.query('NDC>5 & RR<0.5').head()

,EV,AV,RR,NDC,angles


In [20]:
df_combination_rand.RR.min(), df_combination_rand.NDC.max()

(14.837031446912754, 0.879353682820715)

In [21]:
#All combinations within certain angle range
n = 0
df_combination = pd.DataFrame()
angles_com = []
for angle in tqdm(combinations(valid_angles[:11], 10)):
    EV, AV, RR, NDC = data_process(paths, angle)
    #df_combination['angle']=list(angle)
    df_combination.loc[n,'EV']=EV
    df_combination.loc[n,'AV']=AV
    df_combination.loc[n,'RR']=RR
    df_combination.loc[n,'NDC']=NDC
    angles_com.append(angle)
    n += 1

df_combination['angles'] = angles_com

11it [00:00, 21.18it/s]


In [22]:
df_combination.RR.min(), df_combination.NDC.max()

(17.112362822092276, 0.10662566583992683)

In [26]:
slot2_angles_num = [350,50,195,215,335,40,255,115,240,200]

**Calculate ESFQR/ESFQX 2EE**

In [27]:
#calculate ESFQR 2EE
def single_day_data_process(path, angles):
    day = pd.read_csv(path, skiprows=43, usecols= ['Source Slot', 'Job Cycle', 'ESFQR Angle (deg)', 'ESFQR Value (nm)'])
    day = day.dropna()
    day = day[day['Source Slot']==1]
    day = day[(day['Job Cycle']== 1) | (day['Job Cycle']== 2) | (day['Job Cycle']== 3)]
    day = day[day['ESFQR Angle (deg)'].isin(angles)]
    
    X = np.mean(day['ESFQR Value (nm)'])
    
    max_v = day.groupby(by='ESFQR Angle (deg)')['ESFQR Value (nm)'].max()
    min_v = day.groupby(by='ESFQR Angle (deg)')['ESFQR Value (nm)'].min()
    mean_df = day.groupby(by='ESFQR Angle (deg)')['ESFQR Value (nm)'].mean()
    R = np.mean(max_v-min_v)
    
    return X, R, mean_df

In [30]:
#calculate ESFQR 2EE
#ESFQR 2EE
paths = [r'D:\PWG_data\2019-10-1\Gauge-1-2-3-ESFQx_72Sector_2EE-191001132242.csv',
        r'D:\PWG_data\2019-10-2\Gauge-1-2-3-ESFQx_72Sector_2EE-191002102847.csv']
slot2_angles_num = [215,205,210,255,0,75,130,120,210,330]

EV, AV, RR, NDC = data_process(paths, slot2_angles_num)
print('EV = %s; AV = %s; RR = %s; NDC = %s' %(EV, AV, RR, NDC))

EV = 0.19299466666666676; AV = 0.26452834423993843; RR = 0.32744799017263976; NDC = 9.5685295803712


**Calculate ESFQR/ESFQX 1EE**

In [31]:
#ESFQR SHR 1EE
paths = [r'D:\PWG_data\2019-10-1\Gauge-1-2-3-ESFQx_72Sector_1EE_SHR-191001132242.csv',
        r'D:\PWG_data\2019-10-2\Gauge-1-2-3-ESFQx_72Sector_1EE_SHR-191002102847.csv',
        r'D:\PWG_data\2019-10-3\Gauge-1-2-3-ESFQx_72Sector_1EE_SHR-191003095807.csv']

In [38]:
#calculate ESFQR 1EE
def single_day_data_process(path, angles):
    day = pd.read_csv(path, skiprows=43, usecols= ['Source Slot', 'Job Cycle', 'ESFQR Angle (deg)', 'ESFQR Value (nm)'])
    day = day.dropna()
    day = day[day['Source Slot']==5]
    day = day[(day['Job Cycle']== 3) | (day['Job Cycle']== 4) | (day['Job Cycle']== 5)]
    day = day[day['ESFQR Angle (deg)'].isin(angles)]
    
    X = np.mean(day['ESFQR Value (nm)'])
    
    max_v = day.groupby(by='ESFQR Angle (deg)')['ESFQR Value (nm)'].max()
    min_v = day.groupby(by='ESFQR Angle (deg)')['ESFQR Value (nm)'].min()
    mean_df = day.groupby(by='ESFQR Angle (deg)')['ESFQR Value (nm)'].mean()
    R = np.mean(max_v-min_v)
    
    return X, R, mean_df

In [39]:
EV, AV, RR, NDC = data_process(paths, slot2_angles_num)
print('EV = %s; AV = %s; RR = %s; NDC = %s' %(EV, AV, RR, NDC))

EV = 0.24050483333333325; AV = 0.28609866271368195; RR = 0.3737579693642017; NDC = 11.224744773924186


**Finding the best sites for SFQR**

In [32]:
#SFQR std 2EE
paths = [r'D:\PWG_data\2019-10-1\Gauge-1-2-3-SFQx_26x8_2EE-191001132242.csv',
        r'D:\PWG_data\2019-10-2\Gauge-1-2-3-SFQx_26x8_2EE-191002102847.csv',
        r'D:\PWG_data\2019-10-3\Gauge-1-2-3-SFQx_26x8_2EE-191003095807.csv']

In [33]:
def single_day_data_process(path, sites):
    day = pd.read_csv(path, skiprows=43, usecols= ['Source Slot', 'Job Cycle', 'Site Number', 'SFQR Value (nm)'])
    day = day.dropna()
    day = day[day['Source Slot']==2]
    day = day[(day['Job Cycle']== 3) | (day['Job Cycle']== 4) | (day['Job Cycle']== 5)]
    day = day[day['Site Number'].isin(sites)]
    
    X = np.mean(day['SFQR Value (nm)'])
    
    max_v = day.groupby(by='Site Number')['SFQR Value (nm)'].max()
    min_v = day.groupby(by='Site Number')['SFQR Value (nm)'].min()
    mean_df = day.groupby(by='Site Number')['SFQR Value (nm)'].mean()
    R = np.mean(max_v-min_v)
    
    return X, R, mean_df

In [34]:
valid_sites = list(range(1,337))
remove_ls = [1,4,5,10,11,16,17,24,25,32,49,58,59,68,69,78,109,120,121,132,133,144,145,156,157,168,169,180,
             181,192,193,204,205,216,217,228,259,268,269,278,279,288,305,312,313,320,321,326,327,332,333,336]

for num in remove_ls:
    valid_sites.remove(num)

In [35]:
#random sampling
n = 0
df_combination_rand = pd.DataFrame()
sites_com_rand = []
for i in tqdm(range(1*10**3)):
    sites = random.sample(valid_sites,10) 
    EV, AV, RR, NDC = data_process(paths, sites)
    #df_combination_rand.loc[n,'angle']=angle
    df_combination_rand.loc[n,'EV']=EV
    df_combination_rand.loc[n,'AV']=AV
    df_combination_rand.loc[n,'RR']=RR
    df_combination_rand.loc[n,'NDC']=NDC
    sites_com_rand.append(sites)
    n += 1

df_combination_rand['sites'] = sites_com_rand

100%|██████████████████████████████████████| 1000/1000 [00:57<00:00, 17.42it/s]


In [36]:
df_combination_rand.NDC.min(), df_combination_rand.NDC.max()

(2.301183395991168, 22.41037994658251)

In [37]:
df_combination_rand.RR.min(), df_combination_rand.RR.max()

(0.18493283545429837, 0.43710638384318334)

In [38]:
df_combination_rand.to_csv(r'D:\data\1k_SFQR_std_2EE.csv')

In [39]:
best = pd.read_csv(r'D:\PWG_data\slot2_c3-5\1k_SFQR_std_2EE.csv')
best.sort_values('RR').head()

,Unnamed: 0,EV,AV,RR,NDC,sites
3310,3310,0.171332,0.058015,0.180888,15.961569,"[138, 206, 19, 60, 90, 149, 296, 230, 233, 335]"
4745,4745,0.181573,0.021306,0.182818,14.879078,"[277, 238, 179, 219, 258, 293, 114, 117, 301, ..."
483,483,0.179012,0.047146,0.185117,12.873191,"[138, 18, 64, 178, 229, 294, 227, 203, 302, 335]"
803,803,0.184723,0.030296,0.187191,9.191772,"[138, 148, 209, 190, 189, 22, 206, 14, 52, 123]"
3129,3129,0.177634,0.059771,0.187420,16.443978,"[141, 275, 138, 300, 227, 101, 260, 194, 152, ..."


In [40]:
#best_sites = [138,206,19,60,90,149,296,230,233,335]
#best_sites = [277,238,179,219,258,293,114,117,301,151]
best_sites = [277,238,179,219,258,293,114,117,301,151]

len(best_sites)

10

**Calculate SFQx 2EE std**

In [41]:
#SFQX std 2EE
paths = [r'D:\PWG_data\2019-10-1\Gauge-1-2-3-SFQx_26x8_2EE-191001132242.csv',
        r'D:\PWG_data\2019-10-2\Gauge-1-2-3-SFQx_26x8_2EE-191002102847.csv',
        r'D:\PWG_data\2019-10-3\Gauge-1-2-3-SFQx_26x8_2EE-191003095807.csv']

In [42]:
def single_day_data_process(path, sites):
    day = pd.read_csv(path, skiprows=43, usecols= ['Source Slot', 'Job Cycle', 'Site Number', 'SFQX Value (nm)'])
    day = day.dropna()
    day = day[day['Source Slot']==2]
    day = day[(day['Job Cycle']== 3) | (day['Job Cycle']== 4) | (day['Job Cycle']== 5)]
    day = day[day['Site Number'].isin(sites)]
    
    X = np.mean(day['SFQX Value (nm)'])
    
    max_v = day.groupby(by='Site Number')['SFQX Value (nm)'].max()
    min_v = day.groupby(by='Site Number')['SFQX Value (nm)'].min()
    mean_df = day.groupby(by='Site Number')['SFQX Value (nm)'].mean()
    R = np.mean(max_v-min_v)
    
    return X, R, mean_df

In [43]:
EV, AV, RR, NDC = data_process(paths, best_sites)
print('EV = %s; AV = %s; RR = %s; NDC = %s' %(EV, AV, RR, NDC))

EV = 0.05760299999999994; AV = 0.021093633629604137; RR = 0.061343679288904406; NDC = 29.792381891422743


**Calculate SiteNT PV**

In [59]:
#SiteNT std 2EE
paths = [r'D:\PWG_data\2019-10-1\Gauge-1-2-3-siteNT_Front_26x33_2EE-191001132242.csv',
        r'D:\PWG_data\2019-10-2\Gauge-1-2-3-siteNT_Front_26x33_2EE-191002102847.csv',
        r'D:\PWG_data\2019-10-3\Gauge-1-2-3-siteNT_Front_26x33_2EE-191003095807.csv']

In [60]:
def single_day_data_process(path, sites):
    day = pd.read_csv(path, skiprows=41, usecols= ['Source Slot', 'Job Cycle', 'Site Number', 
                                                   'Front Site Based NT PV Value (nm)'])
    day = day.dropna()
    day = day[day['Source Slot']==2]
    day = day[(day['Job Cycle']== 3) | (day['Job Cycle']== 4) | (day['Job Cycle']== 5)]
    day = day[day['Site Number'].isin(sites)]
    
    X = np.mean(day['Front Site Based NT PV Value (nm)'])
    
    max_v = day.groupby(by='Site Number')['Front Site Based NT PV Value (nm)'].max()
    min_v = day.groupby(by='Site Number')['Front Site Based NT PV Value (nm)'].min()
    mean_df = day.groupby(by='Site Number')['Front Site Based NT PV Value (nm)'].mean()
    R = np.mean(max_v-min_v)
    
    return X, R, mean_df

In [65]:
EV, AV, RR, NDC = data_process(paths, best_sites)
print('EV = %s; AV = %s; RR = %s; NDC = %s' %(EV, AV, RR, NDC))

EV = nan; AV = 0; RR = nan; NDC = nan


**GRR Precision with random selected sites (for SFQR/x)**

In [66]:
valid_sites = list(range(1,337))
remove_ls = [1,4,5,10,11,16,17,24,25,32,49,58,59,68,69,78,109,120,121,132,133,144,145,156,157,168,169,180,
             181,192,193,204,205,216,217,228,259,268,269,278,279,288,305,312,313,320,321,326,327,332,333,336]

for num in remove_ls:
    valid_sites.remove(num)
    
sites = random.sample(valid_sites,10)

In [72]:
sites = [310, 231, 37, 273, 117, 110, 66, 39, 22, 214]

In [73]:
paths = [r'D:\PWG_data\2019-10-1\Gauge-1-2-3-SFQx_26x8_2EE-191001132242.csv',
        r'D:\PWG_data\20181014\Gauge-1-2-3-SFQx_26x8_2EE-181014123620.csv']

In [76]:
def single_day_data_process(path, sites):
    day = pd.read_csv(path, skiprows=43, usecols= ['Source Slot', 'Job Cycle', 'Site Number', 
                                                   'SFQX Value (nm)'])
    day = day.dropna()
    day = day[day['Source Slot']==1]
    day = day[(day['Job Cycle']== 1) | (day['Job Cycle']== 2) | (day['Job Cycle']== 3)]
    day = day[day['Site Number'].isin(sites)]
    
    X = np.mean(day['SFQX Value (nm)'])
    
    max_v = day.groupby(by='Site Number')['SFQX Value (nm)'].max()
    min_v = day.groupby(by='Site Number')['SFQX Value (nm)'].min()
    mean_df = day.groupby(by='Site Number')['SFQX Value (nm)'].mean()
    R = np.mean(max_v-min_v)
    
    return X, R, mean_df

In [77]:
EV, AV, RR, NDC = data_process(paths, sites)
print('EV = %s; AV = %s; RR = %s; NDC = %s' %(EV, AV, RR, NDC))

EV = 0.06616960000000001; AV = 0.5547062620072309; RR = 0.5586389290715379; NDC = 1.312825658639488


**GRR Precision with random selected sites (for NT/IPD)**

In [77]:
valid_sites = list(range(1,81))
remove_ls = [1,2,7,8,9,18,29,40,41,52,63,72,73,74,79,80]

for num in remove_ls:
    valid_sites.remove(num)
    
sites = random.sample(valid_sites,10)

In [46]:
sites = [48, 60, 61, 26, 23, 71, 44, 17, 34, 57]

In [69]:
paths = [r'D:\PWG_data\2019-10-1\Gauge-1-2-3-siteNT_Back_26x33_2EE-191001132242.csv',
        r'D:\PWG_data\20181014\Gauge-1-2-3-siteNT_Back_26x33_2EE-181014123620.csv']

In [70]:
def single_day_data_process(path, sites):
    day = pd.read_csv(path, skiprows=41, usecols= ['Source Slot', 'Job Cycle', 'Site Number', 
                                                   'Back Site Based NT PV Value (nm)'])
    day = day.dropna()
    day = day[day['Source Slot']==1]
    day = day[(day['Job Cycle']== 1) | (day['Job Cycle']== 2) | (day['Job Cycle']== 3)]
    day = day[day['Site Number'].isin(sites)]
    
    X = np.mean(day['Back Site Based NT PV Value (nm)'])
    
    max_v = day.groupby(by='Site Number')['Back Site Based NT PV Value (nm)'].max()
    min_v = day.groupby(by='Site Number')['Back Site Based NT PV Value (nm)'].min()
    mean_df = day.groupby(by='Site Number')['Back Site Based NT PV Value (nm)'].mean()
    R = np.mean(max_v-min_v)
    
    return X, R, mean_df

In [71]:
EV, AV, RR, NDC = data_process(paths, sites)
print('EV = %s; AV = %s; RR = %s; NDC = %s' %(EV, AV, RR, NDC))

EV = 0.13647479999999973; AV = 1.1164685946816375; RR = 1.1247788644642245; NDC = 3.53558251816394


In [102]:
sites

[48, 60, 61, 26, 23, 71, 44, 17, 34, 57]

**GRR Precision with random selected sectors (ZDD/IPD)**

In [46]:
valid_angles = [0,22.5,45,67.5,90,112.5,135,157.5,180,202.5,225,247.5,292.5,315,337.5]
 
angles = random.sample(valid_angles,10)
angles = [22.5, 157.5, 90, 315, 225, 135, 337.5, 292.5, 247.5, 0]

In [47]:
paths = [r'D:\PWG_data\20181105\Gauge-1-2-3-IPD_Shape_10term_sector-181105202716.csv',
        r'D:\PWG_data\20190113\Gauge_SHR-1-2-3-IPD_Shape_10term_sector-190113101316.csv']

In [36]:
def single_day_data_process(path, angles):
    day = pd.read_csv(path, skiprows=44, usecols= ['Source Slot', 'Job Cycle', 'Shape Residual X IPD @ (6 & 16) Angle (deg)',
                                                   'Shape Residual X IPD @ (6 & 16) Zone',
                                                   'Shape Residual Y IPD @ (6 & 16) 3Sigma (nm / mm)'])
    day = day.dropna()
    day = day[day['Source Slot']==1]
    day = day[day['Shape Residual X IPD @ (6 & 16) Zone']==1]
    day = day[(day['Job Cycle']== 1) | (day['Job Cycle']== 2) | (day['Job Cycle']== 3)]
    day = day[day['Shape Residual X IPD @ (6 & 16) Angle (deg)'].isin(angles)]
    
    X = np.mean(day['Shape Residual Y IPD @ (6 & 16) 3Sigma (nm / mm)'])
    
    max_v = day.groupby(by='Shape Residual X IPD @ (6 & 16) Angle (deg)')['Shape Residual Y IPD @ (6 & 16) 3Sigma (nm / mm)'].max()
    min_v = day.groupby(by='Shape Residual X IPD @ (6 & 16) Angle (deg)')['Shape Residual Y IPD @ (6 & 16) 3Sigma (nm / mm)'].min()
    mean_df = day.groupby(by='Shape Residual X IPD @ (6 & 16) Angle (deg)')['Shape Residual Y IPD @ (6 & 16) 3Sigma (nm / mm)'].mean()
    R = np.mean(max_v-min_v)
    
    return X, R, mean_df

In [39]:
EV, AV, RR, NDC = data_process(paths, angles)
print('EV = %s; AV = %s; RR = %s; NDC = %s' %(EV, AV, RR, NDC))

EV = 0.15387976799999892; AV = 12.736904747294961; RR = 12.737834255659973; NDC = 4.235039041501878


**GRR Precision with random selected sectors (for ESFQR/x)**

In [190]:
valid_angles = list(range(0,360, 5))
remove_ls = [0,135,225]

for num in remove_ls:
    valid_angles.remove(num)

angles = random.sample(valid_angles,10)

In [88]:
angles = [205, 250, 30, 100, 300, 235, 130, 220, 90, 335]

In [75]:
#ESFQR SHR 1EE
paths = [r'D:\PWG_data\20181105\Gauge-1-2-3-ESFQx_72Sector_1EE_SHR-181105202716.csv',
        r'D:\PWG_data\20190113\Gauge_SHR-1-2-3-ESFQx_72Sector_1EE_SHR-190113101316.csv']

In [78]:
#calculate ESFQR 1EE
def single_day_data_process(path, angles):
    day = pd.read_csv(path, skiprows=43, usecols= ['Source Slot', 'Job Cycle', 'ESFQR Angle (deg)', 'ESFQX Value (nm)'])
    day = day.dropna()
    day = day[day['Source Slot']==1]
    day = day[(day['Job Cycle']== 1) | (day['Job Cycle']== 2) | (day['Job Cycle']== 3)]
    day = day[day['ESFQR Angle (deg)'].isin(angles)]
    
    X = np.mean(day['ESFQX Value (nm)'])
    
    max_v = day.groupby(by='ESFQR Angle (deg)')['ESFQX Value (nm)'].max()
    min_v = day.groupby(by='ESFQR Angle (deg)')['ESFQX Value (nm)'].min()
    mean_df = day.groupby(by='ESFQR Angle (deg)')['ESFQX Value (nm)'].mean()
    R = np.mean(max_v-min_v)
    
    return X, R, mean_df

In [93]:
EV, AV, RR, NDC = data_process(paths, angles)
print('EV = %s; AV = %s; RR = %s; NDC = %s' %(EV, AV, RR, NDC))

EV = 0.08389359999999989; AV = 20.447204963164463; RR = 20.447377067530653; NDC = 0.16805641470057706
